In [1]:
import os
import numpy as np
import pandas as pd
import re

from keras.models import load_model
from keras.preprocessing import image
from keras.applications import Xception

Using TensorFlow backend.


In [2]:
test_folder = "./test/"
feature_size = 299

model_path = "./Xception_whole.h5"
model = load_model(model_path)

#middle_faeture_shape = (10,10,2048)
#X_model = Xception(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
#X_model.load_weights("./xception_weights_notop.h5")

print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               104858112 
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                6156      
Total params: 125,725,748
Trainable params: 125,671,220

In [3]:
def preprocessing(x,output_size = None,pre_model = None):
    if pre_model is None:
        print('preprocessing do nothing')
        return x
    elif output_size is None:
        raise Exception('should use output_size for pre_model')
    else:
        middle_faeture = pre_model.predict(x)
        res = np.reshape(middle_faeture,(1,output_size[0] * output_size[1] * output_size[2]))
        return res

In [4]:
res = []

for f in os.listdir(test_folder):
    img = image.load_img(test_folder+f,target_size=(feature_size,feature_size))
    print('predicting file %s' % (test_folder+f))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    x = preprocessing(x,None,None)
    pre = model.predict(x)
    label_num = np.argmax(pre[0])
    if label_num == 0:
        label = 'norm'
    else:
        label = 'defect'+str(label_num)
    res.append({
        'in':int(f.split('.')[0]),
        'file':f,
        'label':label
    })

predicting file ./test/351.jpg
preprocessing do nothing
predicting file ./test/132.jpg
preprocessing do nothing
predicting file ./test/275.jpg
preprocessing do nothing
predicting file ./test/193.jpg
preprocessing do nothing
predicting file ./test/373.jpg
preprocessing do nothing
predicting file ./test/346.jpg
preprocessing do nothing
predicting file ./test/378.jpg
preprocessing do nothing
predicting file ./test/256.jpg
preprocessing do nothing
predicting file ./test/420.jpg
preprocessing do nothing
predicting file ./test/318.jpg
preprocessing do nothing
predicting file ./test/198.jpg
preprocessing do nothing
predicting file ./test/292.jpg
preprocessing do nothing
predicting file ./test/38.jpg
preprocessing do nothing
predicting file ./test/274.jpg
preprocessing do nothing
predicting file ./test/43.jpg
preprocessing do nothing
predicting file ./test/79.jpg
preprocessing do nothing
predicting file ./test/4.jpg
preprocessing do nothing
predicting file ./test/381.jpg
preprocessing do nothi

predicting file ./test/180.jpg
preprocessing do nothing
predicting file ./test/25.jpg
preprocessing do nothing
predicting file ./test/153.jpg
preprocessing do nothing
predicting file ./test/52.jpg
preprocessing do nothing
predicting file ./test/319.jpg
preprocessing do nothing
predicting file ./test/390.jpg
preprocessing do nothing
predicting file ./test/409.jpg
preprocessing do nothing
predicting file ./test/227.jpg
preprocessing do nothing
predicting file ./test/341.jpg
preprocessing do nothing
predicting file ./test/297.jpg
preprocessing do nothing
predicting file ./test/189.jpg
preprocessing do nothing
predicting file ./test/112.jpg
preprocessing do nothing
predicting file ./test/115.jpg
preprocessing do nothing
predicting file ./test/162.jpg
preprocessing do nothing
predicting file ./test/137.jpg
preprocessing do nothing
predicting file ./test/133.jpg
preprocessing do nothing
predicting file ./test/432.jpg
preprocessing do nothing
predicting file ./test/123.jpg
preprocessing do no

predicting file ./test/433.jpg
preprocessing do nothing
predicting file ./test/282.jpg
preprocessing do nothing
predicting file ./test/11.jpg
preprocessing do nothing
predicting file ./test/161.jpg
preprocessing do nothing
predicting file ./test/414.jpg
preprocessing do nothing
predicting file ./test/15.jpg
preprocessing do nothing
predicting file ./test/119.jpg
preprocessing do nothing
predicting file ./test/160.jpg
preprocessing do nothing
predicting file ./test/26.jpg
preprocessing do nothing
predicting file ./test/290.jpg
preprocessing do nothing
predicting file ./test/48.jpg
preprocessing do nothing
predicting file ./test/434.jpg
preprocessing do nothing
predicting file ./test/82.jpg
preprocessing do nothing
predicting file ./test/33.jpg
preprocessing do nothing
predicting file ./test/168.jpg
preprocessing do nothing
predicting file ./test/202.jpg
preprocessing do nothing
predicting file ./test/276.jpg
preprocessing do nothing
predicting file ./test/353.jpg
preprocessing do nothin

In [5]:
tmp = pd.DataFrame(res)
tmp = tmp.sort_values('in')
df = tmp.drop('in',1)

print(tmp.head())
print(df.head())

      file  in    label
294  0.jpg   0     norm
55   1.jpg   1     norm
166  2.jpg   2  defect4
432  3.jpg   3     norm
16   4.jpg   4  defect2
      file    label
294  0.jpg     norm
55   1.jpg     norm
166  2.jpg  defect4
432  3.jpg     norm
16   4.jpg  defect2


In [6]:
df.to_csv('submit.csv',header=False,index=False)